## Resources

- A **resource** commonly refers to:
 
  - **Memory** is the **working memory capability** of a physical or virtual machine (or a Pod's container) and is expressed in **bytes**.
  - **CPU** is the **compute capability** of a physical or virtual machine (or a Pod's container) and is expressed in **CPU cores**.
    - **GPU** is a similar resource but on a **graphics card** with its own **on-board memory**.
  - **Storage** is the **storage capacity** of a physical or virtual machine (or a Pod's container) and is expressed in **bytes**.
    - Other metrics use for storage, besides capacity, is e.g. latency, throughput, IOPS, utilization, and queue length.
  - **Network** is the **network capacity** of a physical or virtual machine (or a Pod's container) and is expressed in **bits/second**.
    - Other metrics use for network, is e.g. latency (s), throughput (bps), speed (bps), bandwidth (bps), jitter, packet loss, etc.

## Resources in Kubernetes

- A **Node** can be a **Physical Machine**, a **Virtual Machine**, or a **Container**.
  - In a Kubernetes scenario (at least for Cloud Providers), a **Node is a Virtual Machine** unning on a **Physical Machine**.
- A **Physical Machine** has a number of **CPU cores** (an integer value) and **Memory** (bytes).
  - A **Physical Machine** can have multiple **CPUs (sockets)**, where each CPU socker can have multiple **CPU cores** per socket.
- Each **Container** running in a **Pod** on a **Node** utilizes a certain amout of a **Physical Machine's** resources (CPU cores and Memory).
  - If we sum this resource utilization for **Containers** running on a **Node** (**Virtual Machine**) we get the **Node's** utilization.
- For **Nodes** (**Virtual Machines**) and Pod **Containers** we are interested in two aspects of these `resources`:
  - `requests` which is the minimum amount of resources (`cpu` cores and `memory`) that a **Node/Container** requests from the **Physical Machine**.
  - `limits` which is the maximum amount of resources (`cpu` cores and `memory`) that a **Node/Container** is permitted to use.

- Assume we have one **Node** running some **Pods**, each with one **Container**.
  - Here we see the:
    - `resource limits` (maximum allowable `cpu` cores and `memory`) for the **Node**.
    - `resource requests` (minimum `cpu` cores and `memory`) for the **Containers**.

  <img src="../notebook_images/v_node_scaling_before.png" alt="Before Scaling" width="280" height="260" style="margin-bottom:2em">

## Specifying Resource Requests and Limits for a Pod's Containers

<img src="../notebook_images/resources.png" alt="HPA YAML" width="700" height="300" style="margin-bottom:2em; float: right">

- A **Pod** can define <span style="color:#DDDD00;font-weight:bold">resouces</span> for each container, where:
  - <span style="color:#DDDD00;font-weight:bold">requests</span> defines the minimum **cpu** and **memory**.
  - <span style="color:#DDDD00;font-weight:bold">limits</span> defines the maximum **cpu** and **memory**.
- If a **Container**:
  - Exceeds its **memory** <span style="color:#DDDD00;font-weight:bold">limits</span>, it is terminated.
    - i.e. the **kubelet** restarts the **Container**.
  - Approaches its **CPU** <span style="color:#DDDD00;font-weight:bold">limits</span>, it is throttled.
    - i.e. the **Container** can't use more **cpu** than this.
- If a **Container** doesn't specify any <span style="color:#DDDD00;font-weight:bold">resouces</span>:
  - It can use an unlimited amount of **cpu** and **memory**.
  - It might hog all **cpu** and **memory** on the **Node**.
  - It might crash (bring down) the **Node**.

## Ensure a Kubenetes cluster is running

- Use any Kubernetes cluster.
  - In this example, a Minikube cluster with 3 nodes is used: `minikube start --nodes 3`.

## Enable the `metrics-server` Minikube addon (if using Minikube)

- Kubenetes uses the `metrics-server` to monitor `resources`.

In [1]:
!minikube addons enable metrics-server
!minikube addons list

💡  metrics-server is an addon maintained by Kubernetes. For any concerns contact minikube on GitHub.
You can view the list of minikube maintainers at: https://github.com/kubernetes/minikube/blob/master/OWNERS
    ▪ Using image registry.k8s.io/metrics-server/metrics-server:v0.6.4
🌟  The 'metrics-server' addon is enabled
|-----------------------------|----------|--------------|--------------------------------|
|         ADDON NAME          | PROFILE  |    STATUS    |           MAINTAINER           |
|-----------------------------|----------|--------------|--------------------------------|
| ambassador                  | minikube | disabled     | 3rd party (Ambassador)         |
| auto-pause                  | minikube | disabled     | minikube                       |
| cloud-spanner               | minikube | disabled     | Google                         |
| csi-hostpath-driver         | minikube | disabled     | Kubernetes                     |
| dashboard                   | minikube |

## Install the Metrics Server (if not using Minikube)

- To check if the Metrics Server is installed in your cluster, look for a pod called `metrics-server` in the `kube-system` namespace
  -  `kubectl get po -n kube-system`
- To install the Metrics Server
  - Download the YAML file `components.yaml` from `https://github.com/kubernetes-sigs/metrics-server/releases`
  - Edit the `components.yaml` file and add the parameter `- --kubelet-insecure-tls` to the `Deployment` section.
    - This enables the Metrics Server to run if TLS is not configured.
  - Install the Metrics Server with `kubectl apply -f components.yaml`.

```bash
# components.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  labels:
    k8s-app: metrics-server
  name: metrics-server
  namespace: kube-system
spec:
  selector:
    matchLabels:
      k8s-app: metrics-server
  strategy:
    rollingUpdate:
      maxUnavailable: 0
  template:
    metadata:
      labels:
        k8s-app: metrics-server
    spec:
      containers:
      - args:
        - --cert-dir=/tmp
        - --secure-port=443
        - --kubelet-preferred-address-types=InternalIP,ExternalIP,Hostname
        - --kubelet-use-node-status-port
        - --kubelet-insecure-tls # <-------------------------------------------- Add this line
        - --metric-resolution=15s
        image: k8s.gcr.io/metrics-server/metrics-server:v0.5.0
```

In [ ]:
!kubectl apply -f manifests/components.yaml

## Let's list the current CPU and Memory utilization for every node

- **Note:** It might take a minute before the `metrics-server` has collected enough metrics for the nodes.
- CPU is measured in **cpu units**.
  - **1** CPU unit is equivalent to either:
    - 1 physical CPU core, if the machine is a physical host.
    - 1 virtual core, if the machine is a virtual machine (or container) running inside a physical machine.
  - Fractional values, e.g. **0.5**, means half as much CPU time is used compared to using **1.0** CPU.
    - For CPU resource units, the quantity expression **0.1** is equivalent to the expression **100m**, which can be read as "one hundred millicpu".
      - Some people say "one hundred millicores", and this is understood to mean the same thing.
  - CPU resources are always specified as an absolute amount of resource, never as a relative amount.
    - E.g. **500m** represents the same amount of computing power (half a CPU core).
      - Irrespective if the pysical machine has a single-core, dual-cores, or 48-cores.
- Memory is measured in **bytes**.
  - Memory can be expressed as a plain integer or as a fixed-point number using one of the quantity suffixes:
    - **E**, **P**, **T**, **G**, **M**, **k** (**E**ta bytes, **P**eta bytes, **T**era bytes, **G**iga bytes, **M**ega bytes, **K**ilo bytes), e.g. 1**K** is 1 kilobyte = 1000 bytes.
    - You can also use the power-of-two equivalents: **Ei**, **Pi**, **Ti**, **Gi**, **Mi**, **Ki**, e.g 1**Ki** is 1 kebibyte = 1024 bytes.
    - You can also use the suffix **m** for millibytes, e.g. 1000**m** is 1 millibyte = 1 byte.
    - E.g., the following represent roughly the same value: 128974848, 129e6, 129**M**, 123**Mi**, 128974848000**m**
  
  In this example, the three Minikube nodes are currently using the following CPU and Memory resources:
  - The `minikube` node is using:
    - `157m` (0.176) CPU cores (`0%`) of total available CPU resources.
    - `968Mi` (762 memibytes) of Memory (`6%`) of total available memory resources.
  - The `minikube-m02` node is using:
    - `43m` (0.046) CPU cores (`0%`) of total available CPU resources.
    - `269Mi` (268 memibytes) of Memory (`1%`) of total available memory resources.
  - The `minikube-m03` node is using:
    - `35m` (0.039) CPU cores (`0%`) of total available CPU resources.
    - `225Mi` (226 memibytes) of Memory (`1%`) of total available memory resources.

  ```bash
  NAME           CPU(cores)   CPU%   MEMORY(bytes)   MEMORY%   
  minikube       157m         0%     968Mi           6%
  minikube-m02   43m          0%     269Mi           1%
  minikube-m03   35m          0%     225Mi           1%
  ```

In [4]:
!kubectl top nodes

NAME           CPU(cores)   CPU%   MEMORY(bytes)   MEMORY%   
minikube       157m         0%     968Mi           6%        
minikube-m02   43m          0%     269Mi           1%        
minikube-m03   35m          0%     225Mi           1%        


## Create Pods

- The following Pod's use stress images (used to stress the cpu/memory in a container).
  - All Pod definitions have  `replicas` set to `1`.

- The Pod definitions are in the YAML files as below:
  - CPU tests:
    - `manifests/within-node-cpu-limit.yaml`
      - the container's cpu (and memory) doesn't exceed the node's cpu and memory
      - the container tries to use 2 cpu cores, but is limited (throttled) to 0.030 cpu cores
      - the Pod will run, although the microservice might run slower than expected

      ```bash
      resources:
        requests:            # minimum cpu is 0.015 cores, minimum memory is 100 mebibytes
          cpu: 15m
          memory: 100Mi
        limits:              # maximum cpu is 0.030 cores, minimum memory is 200 mebibytes
          cpu: 30m
          memory: 200Mi
      args: ["-cpus", "2"]   # the pod's container actaully uses 2 cores (the stress test)
      ```

    - `manifests/over-node-cpu-limit.yaml`
      - the container's requested cpu exceeds the node's cpu limit
      - the Pod will not run if no node can accomodate the contaienr's resource demands
        - the Pods `STATE` will be `pending` (util an appropriate node becomes available)

      ```bash
      resources:
        requests:            # minimum cpu is 100 cores, minimum memory is 100 mebibytes
          cpu: 100000m
          memory: 100Mi
        limits:              # maximum cpu is 100 cores, minimum memory is 200 mebibytes
          cpu: 100000m
          memory: 200Mi
      args: ["-cpus", "2"]   # the pod's container actually uses 2 cores (the stress test)
      ```

  - Memory tests: 
    - `manifests/within-container-memory-limit.yaml`
      - the container's memory utilization (150M) is within its memory limit (200Mi)
      - the Pod will run as normal without any hicks

      ```bash
      resources:
        requests:            # minimum cpu is 0.015 cores, minimum memory is 100 mebibytes
          cpu: 15m
          memory: 100Mi
        limits:              # maximum cpu is 0.030 cores, minimum memory is 200 mebibytes
          cpu: 30m
          memory: 200Mi
      command: [stress]      # the pod's container actually uses 150 megabytes (the stress test)
      args: ["--vm", "1", "--vm-bytes", "150M", "--vm-hang", "1"]
      ```
    
    - `manifests/over-container-memory-limit.yaml`
      - the container's memeory utilization (250M) exceeds its memory limit (100Mi)
      - the kubelet running on the node will restart the Pod
        - first, the Pod's `STATE` will be `OOMKilled` (Out Of Memory, so the Pod was killed)
        - then the Pod will be restarted by the kubelet
        - finally, the Pod's `STATE` will be `CrashLoopBackOff` (keeps crashing, so not restarted for a while)

      ```bash
      resources:
        requests:            # minimum cpu is 0.015 cores, minimum memory is 50 mebibytes
          cpu: 15m
          memory: 50Mi
        limits:              # maximum cpu is 0.030 cores, minimum memory is 100 mebibytes
          cpu: 30m
          memory: 100Mi
      command: [stress]      # the pod's container actually uses 250 megabytes (the stress test)
      args: ["--vm", "1", "--vm-bytes", "250M", "--vm-hang", "1"]
      ```
  
    - `manifests/over-node-memory-limit.yaml`
      - the container's requested memory exceeds the node's available memory
      - the Pod will not run if no node can accomodate the contaienr's resource demands
        - the Pods `STATE` will be `pending` (util an appropriate node becomes available)

      ```bash
      resources:
        requests:            # minimum cpu is 0.015 cores, minimum memory is 1000 gibibytes (1 tebibyte)
          cpu: 15m
          memory: 1000Gi
        limits:              # maximum cpu is 0.030 cores, minimum memory is 1000 gibibytes (1 tebibyte)
          cpu: 30m
          memory: 1000Gi
      command: [stress]      # the pod's container actually uses 150 megabytes (the stress test)
      args: ["--vm", "1", "--vm-bytes", "150M", "--vm-hang", "1"]
      ```

In [5]:
!kubectl apply -f manifests/within-container-memory-limit.yaml
!kubectl apply -f manifests/over-container-memory-limit.yaml
!kubectl apply -f manifests/over-node-memory-limit.yaml

!kubectl apply -f manifests/within-node-cpu-limit.yaml
!kubectl apply -f manifests/over-node-cpu-limit.yaml

pod/within-container-memory-limit created
pod/over-container-memory-limit created
pod/over-node-memory-limit created
pod/within-node-cpu-limit created
pod/over-node-cpu-limit created


## List Pods

- We see that the `STATUS` is:
  - `OOMKilled` for the `over-container-memory-limit` pod.
    - this pod was killed since it's container's memory utilization excceded it memory limit.
    - the pod will be restarted, but eventually end up with a `STATUS` of `CrashLoopBackOff`.
  - `Pending` for the `over-node-cpu-limit` and `over-node-memory-limit` pods.
    - the `kube-scheduler` can't find an appropriate node that can accommodate the container's resource demands. 
  - `Running` for the `within-container-memory-limit` and `within-node-cpu-limit` pods.
    - the first Pod's container's memory utilization is within its memory limit.
    - the second Pod's cpu utilization is throttled (capped) to its cpu limit.

**Note:**
- A container that exceeds it's memory limit (`OOMKilled`) is restarted by the `kubelet`.
- A container that tries to exceed it's cpu limit is throttled (its cpu utilization is capped) by the `kubelet`.
- A container that demands more cpu or memory available on any node, will be in a `Pedning` state.

You can run the command `kubectl get pods -o wide --watch` in a separate terminal to see that pod states chaning.

In [9]:
!kubectl get pods -o wide

NAME                            READY   STATUS      RESTARTS      AGE   IP           NODE           NOMINATED NODE   READINESS GATES
over-container-memory-limit     0/1     OOMKilled   3 (35s ago)   61s   10.244.2.2   minikube-m03   <none>           <none>
over-node-cpu-limit             0/1     Pending     0             60s   <none>       <none>         <none>           <none>
over-node-memory-limit          0/1     Pending     0             60s   <none>       <none>         <none>           <none>
within-container-memory-limit   1/1     Running     0             61s   10.244.1.3   minikube-m02   <none>           <none>
within-node-cpu-limit           1/1     Running     0             60s   10.244.2.3   minikube-m03   <none>           <none>


## Describe the `over-node-memory-limit` Pod

- We see the `over-node-memory-limit` Pod has a `FailedScheduling` event (bottom of the listing).
  - There is insufficient memory on any node to run the pod (who's container is demanding too much memory).

  ```bash
  Events:
    Type     Reason            Age   From               Message
    ----     ------            ----  ----               -------
    Warning  FailedScheduling  13s   default-scheduler  0/3 nodes are available: 3 Insufficient memory.
  ```

In [10]:
!kubectl describe pod over-node-memory-limit

Name:             over-node-memory-limit
Namespace:        default
Priority:         0
Service Account:  default
Node:             <none>
Labels:           <none>
Annotations:      <none>
Status:           Pending
IP:               
IPs:              <none>
Containers:
  over-node-memory-limit:
    Image:      polinux/stress
    Port:       <none>
    Host Port:  <none>
    Command:
      stress
    Args:
      --vm
      1
      --vm-bytes
      150M
      --vm-hang
      1
    Limits:
      cpu:     30m
      memory:  1000Gi
    Requests:
      cpu:        15m
      memory:     1000Gi
    Environment:  <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from kube-api-access-vv4cc (ro)
Conditions:
  Type           Status
  PodScheduled   False 
Volumes:
  kube-api-access-vv4cc:
    Type:                    Projected (a volume that contains injected data from multiple sources)
    TokenExpirationSeconds:  3607
    ConfigMapName:           kube-root-ca.crt
    ConfigM

## Describe the `over-node-cpu-limit` Pod

- We see the `over-node-cpu-limit` Pod has a `FailedScheduling` event (bottom of the listing).
  - There is insufficient cpu on any node to run the pod (who's container is demanding too much cpu).

  ```bash
  Events:
    Type     Reason            Age   From               Message
    ----     ------            ----  ----               -------
    Warning  FailedScheduling  14s   default-scheduler  0/3 nodes are available: 3 Insufficient cpu.
  ```

In [11]:
!kubectl describe pod over-node-cpu-limit

Name:             over-node-cpu-limit
Namespace:        default
Priority:         0
Service Account:  default
Node:             <none>
Labels:           <none>
Annotations:      <none>
Status:           Pending
IP:               
IPs:              <none>
Containers:
  over-node-cpu-limit:
    Image:      vish/stress
    Port:       <none>
    Host Port:  <none>
    Args:
      -cpus
      2
    Limits:
      cpu:     100
      memory:  200Mi
    Requests:
      cpu:        100
      memory:     100Mi
    Environment:  <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from kube-api-access-wjjxm (ro)
Conditions:
  Type           Status
  PodScheduled   False 
Volumes:
  kube-api-access-wjjxm:
    Type:                    Projected (a volume that contains injected data from multiple sources)
    TokenExpirationSeconds:  3607
    ConfigMapName:           kube-root-ca.crt
    ConfigMapOptional:       <nil>
    DownwardAPI:             true
QoS Class:                   B

## Get resource utilization information for the Nodes and Pods.

- For the Nodes we see that: 
  - The `minikube` node is using:
    - `175m` `cpu` cores (`1%` of available physical `cpu` cores)
    - `991Mi` `memory` (`6%` of available physical `memory`)
  - The `minikube-m02` node is using:
    - `82m` `cpu` cores (`0%` of available physical `cpu` cores)
    - `445Mi` `memory` (`2%` of available physical `memory`)
  - The `minikube-m03` node is using:
    - `121m` `cpu` cores (`0%` of available physical `cpu` cores)
    - `265Mi` `memory` (`1%` of available physical `memory`)
  
- For the Pods we see that:
  - The `within-container-memory-limit` pod is using:
    - `13m` `cpu` cores
    - `150Mi` `memory`
  - The `within-node-cpu-limit` pod is using:
    - `30m` `cpu` cores
    - `1Mi` `memory`

In [12]:
!kubectl top nodes
!kubectl top pods

NAME           CPU(cores)   CPU%   MEMORY(bytes)   MEMORY%   
minikube       175m         1%     991Mi           6%        
minikube-m02   82m          0%     445Mi           2%        
minikube-m03   121m         0%     265Mi           1%        
NAME                            CPU(cores)   MEMORY(bytes)   
within-container-memory-limit   13m          150Mi           
within-node-cpu-limit           30m          1Mi             


## Delete the Pods

In [13]:
!kubectl delete -f manifests/within-container-memory-limit.yaml --grace-period=0 --force
!kubectl delete -f manifests/over-container-memory-limit.yaml --grace-period=0 --force
!kubectl delete -f manifests/over-node-memory-limit.yaml --grace-period=0 --force

!kubectl delete -f manifests/within-node-cpu-limit.yaml --grace-period=0 --force
!kubectl delete -f manifests/over-node-cpu-limit.yaml --grace-period=0 --force

pod "within-container-memory-limit" force deleted
pod "over-container-memory-limit" force deleted
pod "over-node-memory-limit" force deleted
pod "within-node-cpu-limit" force deleted
pod "over-node-cpu-limit" force deleted


## Disable the `metrics-server` Minikube addon (if using Minikube)

In [14]:
!minikube addons disable metrics-server
!minikube addons list

🌑  "The 'metrics-server' addon is disabled
|-----------------------------|----------|--------------|--------------------------------|
|         ADDON NAME          | PROFILE  |    STATUS    |           MAINTAINER           |
|-----------------------------|----------|--------------|--------------------------------|
| ambassador                  | minikube | disabled     | 3rd party (Ambassador)         |
| auto-pause                  | minikube | disabled     | minikube                       |
| cloud-spanner               | minikube | disabled     | Google                         |
| csi-hostpath-driver         | minikube | disabled     | Kubernetes                     |
| dashboard                   | minikube | disabled     | Kubernetes                     |
| default-storageclass        | minikube | disabled     | Kubernetes                     |
| efk                         | minikube | disabled     | 3rd party (Elastic)            |
| freshpod                    | minikube | disa

## Uninstall the Metrics Server (if not using Minikube)

- To uninstall the Metrics Server run the command: `kubectl delete -f components.yaml`

In [ ]:
!kubectl delete -f manifests/components.yaml

## Delete the Cluster

- Delete your specific Kubernetes cluster.
  - In this example, a Minikube cluster is used, so the command is:

  ```bash
  minikube stop
  minikube delete
  ```